## Overview

#### written on 10/01/24

#### ~~Following tasks had all been accomplished.~~

separate game_version column into version_1 and version_2

game_version has been a big obstacle, because I want to use it as number,
while the comparison doesn't work as intended in float (version 14.9 is before 14.11, but 14.9 > 14.11)
also, if it is treated as float, 14.10 becomes 14.1 (already some records are spoiled)

Therefore, separate the column.
for spoiled column, since there is no way to separate them out (14.1 and 14.10 are both recorded as 14.10),
call the api again and save the right result.
for non-spoiled column (version_2 being non-zero for all digits), just separate them into version_1 and version_2 and save them to save api calls (14.16 -> version_1 = 14, version_2 = 16)

-> something must have gone wrong, just update all with api calls

matches (fill)
- version_1 (smallint)
- version_2 (smallint)

- remove:
    - game_version from matches

## Import

In [1]:
import time
import importlib
import api_caller
import db_connector

## API call setting

In [2]:
# Reload the module to ensure the latest changes are reflected
importlib.reload(api_caller)

caller = api_caller.APICaller()

# Test your function call
# result = caller.get_account_from_riot_id("NoMatterWhoUR", "NA1")
# print(result)

## DB connection setting

In [3]:
# Reload the module to ensure the latest changes are reflected
importlib.reload(db_connector)

connector = db_connector.DBConnector()

## Query preparation

In [4]:
# matches

'''
original query:
INSERT INTO matches (match_id)
VALUES (%s)
ON CONFLICT (match_id)
DO UPDATE SET version_1 = %s, version_2 = %s
'''

matches_query = {
    "INSERT": "INTO matches",
    "ON CONFLICT": "(match_id)",
    "DO UPDATE SET": ""
}

matches_col = ["match_id"]
matches_update_col = ["version_1", "version_2"]

## Main code

In [5]:
# filter by game_start not filled in the future
# "select match_id from matches where game_start IS NOT NULL"
query = {
    "SELECT": "match_id",
    "FROM": "matches"
}
matches = connector.select_query(query)
# call 100 matches at a time
print(len(matches))

1629


In [6]:
# match = (match_id, game_version as float)

for idx, match in enumerate(matches):


    data = caller.get_match_from_match_id(match[0])

    version = data["info"]["gameVersion"].split(".")

    matches_val = (
        match[0],
        version[0],
        version[1]
    )
    # print(matches_val)
    connector.insert_query(matches_query, matches_col, matches_val, matches_update_col)

    if (idx + 1) % 10 == 0:
        print(idx + 1, " matches have been updated")

10  matches have been updated
20  matches have been updated
30  matches have been updated
40  matches have been updated
50  matches have been updated
60  matches have been updated
70  matches have been updated
80  matches have been updated
90  matches have been updated
have called 100 apis: 2 min break starts
100  matches have been updated
110  matches have been updated
120  matches have been updated
130  matches have been updated
140  matches have been updated
150  matches have been updated
160  matches have been updated
170  matches have been updated
180  matches have been updated
190  matches have been updated
have called 200 apis: 2 min break starts
200  matches have been updated
210  matches have been updated
220  matches have been updated
230  matches have been updated
240  matches have been updated
250  matches have been updated
260  matches have been updated
270  matches have been updated
280  matches have been updated
290  matches have been updated
have called 300 apis: 2 min 

In [7]:
print("commited and ended")
connector.commit()
connector.end()

commited and ended
